In [1]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
client = OpenAI()

In [18]:
# Create the assistant
assistant = client.beta.assistants.create(
    name="Policy Explainer",
    instructions="You answer questions about company rules based on your knowledge of the company policy Markdown file.",
    tools = [{'type':'file_search'}],
    model = 'gpt-4o'
)

In [19]:
assistant.id

'asst_NLEskiKYda13bLcJtKNeZ1FV'

In [50]:
# Create a vector store caled "Company Policies"
vector_store = client.beta.vector_stores.create(name="Company Policies")
 
# Ready the files for upload to OpenAI
file_paths = ["policies/company-policy.md"]
file_streams = [open(path, "rb") for path in file_paths]
 
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [51]:
# Update the assistant with the vector store id(s)
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

In [ ]:
# Add a new file to the vector store, this time in a PDF format. Note, you don't need to update the assistant again
# as it's referring to the vector store id which has not changed.
file_02_path = "policies/health-and-safety-policy.pdf"
file_02_stream = open(file_02_path, "rb")

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=[file_02_stream]
)

In [89]:
# Create a thread
thread = client.beta.threads.create()

In [90]:
thread.id

'thread_upOuLEkNsv78XgfY0YpXKe91'

In [88]:
client.beta.threads.delete('thread_oLPlBIgArf987NnlQX0Wim2q')

ThreadDeleted(id='thread_oLPlBIgArf987NnlQX0Wim2q', deleted=True, object='thread.deleted')

In [91]:
# Create our first message. The answer to this question can be found in the PDF document which was uploaded later.
message_1 = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = 'user',
    content = 'Can I bring my pet cat to the work?'
)

In [92]:
# Create a run for the thread
run = client.beta.threads.runs.create(
    assistant_id = assistant.id,
    thread_id = thread.id)

In [93]:
run.status

'queued'

In [94]:
# Retrieve a run
run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id)

In [95]:
run.status

'completed'

In [96]:
messages = client.beta.threads.messages.list(
    thread_id = thread.id
)

In [97]:
for message in messages:
    print(message)
    print('\n')

Message(id='msg_SLDLAr6niK9mlwSpUSclqv64', assistant_id='asst_NLEskiKYda13bLcJtKNeZ1FV', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=656, file_citation=FileCitation(file_id='file-7eo6hweJlOcWgW7F0VZ5bBSj'), start_index=644, text='【4:0†source】', type='file_citation')], value='Yes, you can bring your pet cat to work, provided certain conditions are met. According to the Specialty Cakes Facility Health and Safety Policy:\n\n1. Pets must be well-behaved, healthy, and up-to-date with vaccinations.\n2. They must be supervised at all times by their owners.\n3. Pets are not allowed in food preparation or storage areas.\n4. Owners must clean up after their pets immediately and ensure the pet is clean and free from parasites.\n5. Pets must be kept on a leash or in a carrier.\n6. Any pet exhibiting aggressive behavior will be asked to leave the premises.\n7. Owners are responsible for any damage or injury caused by their pet

In [98]:
messages.data[0].content[0].text.value


'Yes, you can bring your pet cat to work, provided certain conditions are met. According to the Specialty Cakes Facility Health and Safety Policy:\n\n1. Pets must be well-behaved, healthy, and up-to-date with vaccinations.\n2. They must be supervised at all times by their owners.\n3. Pets are not allowed in food preparation or storage areas.\n4. Owners must clean up after their pets immediately and ensure the pet is clean and free from parasites.\n5. Pets must be kept on a leash or in a carrier.\n6. Any pet exhibiting aggressive behavior will be asked to leave the premises.\n7. Owners are responsible for any damage or injury caused by their pets【4:0†source】.'

In [99]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id = thread.id,
    run_id = run.id
)

In [100]:
for step in run_steps:
    print(step.step_details)
    print('\n')

MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_SLDLAr6niK9mlwSpUSclqv64'), type='message_creation')


ToolCallsStepDetails(tool_calls=[FileSearchToolCall(id='call_nhD7tCRoA3Jif2iGgVaHcQev', file_search={}, type='file_search')], type='tool_calls')




In [101]:
files = client.beta.vector_stores.files.list(
    vector_store_id=vector_store.id
)

In [102]:
files.data

[VectorStoreFile(id='file-7eo6hweJlOcWgW7F0VZ5bBSj', created_at=1718443070, last_error=None, object='vector_store.file', status='completed', usage_bytes=4563, vector_store_id='vs_p3He4Gl0XY7FcOmORnqnk2VA', chunking_strategy=ChunkingStrategyStatic(static=ChunkingStrategyStaticStatic(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static')),
 VectorStoreFile(id='file-1UVS5tbDzkemXgq5LckqMHX0', created_at=1718442940, last_error=None, object='vector_store.file', status='completed', usage_bytes=6187, vector_store_id='vs_p3He4Gl0XY7FcOmORnqnk2VA', chunking_strategy=ChunkingStrategyStatic(static=ChunkingStrategyStaticStatic(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'))]

In [21]:
# Delete the assistant:
response = client.beta.assistants.delete(assistant.id)

In [22]:
response

AssistantDeleted(id='asst_HYSt1vSEPMsIFUrvgvn4bqVy', deleted=True, object='assistant.deleted')